In [ ]:
import os
import shutil
import random
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [23]:
import keras
import keras.applications
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [ ]:
# os.path.exists() – Returns True if path or directory does exists.
# os.path.isfile() – Returns True if path is File.
# os.path.isdir() – Returns True if path is Directory.
# pathlib.Path.exists() – Returns True if path or directory does exists. (In Python 3.4 and above
# shutil.copyfile(src, dst)
# shutil.rmtree('dir_path')
# os.mkdir(path)

In [12]:
def read_image(image_path,size=(224,224)):
  image = cv2.imread(image_path)
  image = cv2.resize(image,size)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  return image

def plot_image(img):
  plt.imshow(img)
  plt.show()



upload dataset using kaggle


In [ ]:
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d yousiftito/cv-2023-sc-dataset
! unzip /content/cv-2023-sc-dataset

## Creat train and validation folders

In [14]:
if os.path.exists('/content/images'):
  shutil.rmtree('/content/images')
os.mkdir('/content/images')
persons=os.listdir('/content/CV_2023_SC_Dataset')
for person in persons:
  imgs_path='/content/CV_2023_SC_Dataset/'+person+'/'+'Train/'

  for img in os.listdir(imgs_path):
    if not img.__contains__('.png'):continue
    src=imgs_path+img
    dst='/content/images/'+img
    shutil.copyfile(src, dst)

print(len(os.listdir('/content/images')))

200


In [15]:
 def split_dataset(directory, split=1):
    folders = os.listdir(directory)
    num_train = int(len(folders)*split)

    random.shuffle(folders)

    train_list, test_list = [], []

    for folder in folders[:num_train]:
        train_list.append(folder)

    for folder in folders[num_train:]:
        test_list.append(folder)

    return train_list, test_list

train_list, val_list = split_dataset('/content/images', split=.8)
print(len(train_list))
print(len(val_list))


160
40


In [16]:
if os.path.exists('/content/train'):
  shutil.rmtree('/content/train')
c={'aa':1}
os.mkdir('/content/train')
for img in train_list:
  catoeg=img.split('_')[0]
  if catoeg not in c.keys():
    os.mkdir('/content/train/'+catoeg)
    c[catoeg]=1


  src='/content/images/'+img
  dst='/content/train/'+catoeg+'/'+img
  shutil.copyfile(src, dst)

c={}
if os.path.exists('/content/val'):
  shutil.rmtree('/content/val')
os.mkdir('/content/val')
# shutil.move("path/to/current/file.foo", "path/to/new/destination/for/file.foo")


for img in val_list:

  catoeg=img.split('_')[0]
  if catoeg not in c.keys():
    os.mkdir('/content/val/'+catoeg)
    c[catoeg]=1

  src='/content/images/'+img
  dst='/content/val/'+catoeg+'/'+img
  shutil.copyfile(src, dst)




# if os.path.exists('/content/val'):
#   shutil.rmtree('/content/val')
# os.mkdir('/content/val')

In [ ]:
img=read_image('/content/images/personA_1.png')
plot_image(img)
img=read_image('/content/images/personB_1.png')
plot_image(img)
img=read_image('/content/images/personC_1.png')
plot_image(img)
img=read_image('/content/images/personD_1.png')
plot_image(img)
img=read_image('/content/images/personE_1.png')
plot_image(img)

# Creat test folder


In [ ]:
!kaggle datasets download -d yousiftito/test-images

In [ ]:
import cv2
calsses=['personA','personB','personC','personD','personE']
tst_images=[]
labels=[]
if os.path.exists('/content/test_images'):
  shutil.rmtree('/content/test_images')
os.mkdir('/content/test_images')
persons=os.listdir('/content/CV_2023_SC_Dataset')
for person in persons:
  imgs_path='/content/CV_2023_SC_Dataset/'+person+'/'+'Test/'
  os.mkdir('/content/test_images/'+person.split('_')[0])###
  print(person.split('_')[0])
  for img in os.listdir(imgs_path):
    if not img.__contains__('.png'):continue
    src=imgs_path+img
    image=read_image(src)
    tst_images.append(image)
    labels.append(calsses.index(person.split('_')[0]))
    dst='/content/test_images/'+person.split('_')[0]+'/'+img
    shutil.copyfile(src, dst)

print(len(os.listdir('/content/test_images')))

In [ ]:
import numpy as np
print(tst_images[0].shape)
t=np.array(tst_images).reshape(-1,224,224,3)

print(len(tst_images))
print(len(labels))
print(labels)


# Data generator


In [20]:
n=5
image_size=224
batch_train_size=160
batch_val_size=40

In [ ]:
data_generator=ImageDataGenerator()
# categorical_crossentropy
train_generator = data_generator.flow_from_directory('/content/train', batch_size = batch_train_size, class_mode = 'categorical', target_size = (image_size, image_size))

# Flow validation images in batches of 20 using test_datagen generator
val_generator = data_generator.flow_from_directory( '/content/val',  batch_size = batch_val_size, class_mode = 'categorical', target_size = (image_size, image_size))

# test_generator = data_generator.flow_from_directory( '/content/test_images',  batch_size = batch_val_size, class_mode = 'categorical', target_size = (image_size, image_size))

# print(type(train_generator))
# print(len(train_generator))
# gg=train_generator.next()
# print((gg[0].shape))
# print((gg[1].shape))

# Creat Model

In [ ]:
# from keras.applications.mobilenet_v3 import models
from keras.layers import Dense, Activation, Flatten
from keras import *
import tensorflow as tf
model=Sequential()
model.add(VGG16(include_top=False,pooling='avg',weights='imagenet',))

for layer in model.layers:
    layer.trainable = False

x = keras.layers.Flatten()(model.output)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

x = layers.Dense(5, activation='sigmoid')(x)

model = tf.keras.models.Model(model.input, x)



In [ ]:
model.compile(optimizer ='adam', loss = 'binary_crossentropy',metrics = ['acc'])
vgghist = model.fit(train_generator, validation_data = val_generator, steps_per_epoch = 1, epochs = 10)


In [ ]:
prediction=model.predict(t)


In [28]:
col2=[]
for x in prediction :
    col2.append(np.argmax(x))

predicted=np.array(col2)
actual=np.array(labels)



# Accurecy

In [ ]:
from sklearn import metrics
# confusion_matrix = metrics.confusion_matrix(actual, predicted)
Accuracy = metrics.accuracy_score(actual, predicted)
print(Accuracy)


In [30]:
model.save('model/model.tfl')

In [ ]:
!zip -r /content/model_zipped.zip /content/model

In [ ]:
from google.colab import files

files.download('/content/model_zipped.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>